In [49]:
# #Reuired installation
# !pip install requests beautifulsoup4
# !pip install pypdf

# !pip install pypdf nltk beautifulsoup4 requests
# !pip install nltk requests beautifulsoup4



In [50]:
# #Required imports
# import requests
# from bs4 import BeautifulSoup
# from pypdf import PdfReader
# import os
# import re
# from collections import defaultdict
# import nltk
# nltk.download('punkt')




In [51]:
ALL_PLANT_DISEASE_KEYWORDS = [
    "plant",
    "disease",
    "diseases",
    "detection",
    "diagnosis",
    "pathogen",
    "pathogens",
    "symptom",
    "symptoms",
    "leaf",
    "leaves",
    "root",
    "crop",
    "loss",
    "necrosis",
    "spot",
    "spots",
    "fungi",
    "fungal",
    "bacterial",
    "virus",
    "viral",
    "nematode",
    "microscopy",
    "laboratory",
    "molecular",
    "pcr",
    "image",
    "learning",
    "model"
]


In [52]:
# STOPWORDS = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to', 'for', 'of', 'with','by','from'}
# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r"[^a-z\s]", " ", text)
#     words = text.split()
#     words = [w for w in words if w not in STOPWORDS]
#     return words

In [53]:
# def load_pdfs_as_units(pdf_paths):
#     units = []

#     for article_id, pdf_path in enumerate(pdf_paths, start=1):
#         reader = PdfReader(pdf_path)

#         for page_number, page in enumerate(reader.pages, start=1):
#             raw_text = page.extract_text()

#             if raw_text and raw_text.strip():
#                 cleaned_words = clean_text(raw_text)

#                 if cleaned_words:
#                     unit = {
#                         "article_id": article_id,
#                         "article_name": os.path.basename(pdf_path),
#                         "page_number": page_number,
#                         "words": cleaned_words
#                     }
#                     units.append(unit)

#     return units


In [54]:
# from collections import defaultdict

# def build_inverted_index_by_document(units):
#     """
#     Build inverted index:
#     term -> list of document IDs (no duplicates, no page numbers)
#     """


#     index = defaultdict(set)  # set to avoid duplicates

#     for unit in units:
#         doc_id = unit["article_name"]
#         unique_words = set(unit["words"])

#         for word in unique_words:
#             index[word].add(doc_id)

#     # convert sets to sorted lists
#     final_index = {
#         term: sorted(list(doc_ids))
#         for term, doc_ids in index.items()
#     }

#     return final_index


In [55]:
# pdf_files = [
#     "Article_1.pdf",
#     "Article_2.pdf",
#     "Article_3.pdf",
#     "Article_4.pdf",
#     "Article_5.pdf"
# ]

# units = load_pdfs_as_units(pdf_files)
# inverted_index = build_inverted_index_by_document(units)



In [56]:
# import re
# def index_words(soup):
#   index = {}
#   words = re.findall(r'\w+', soup.get_text())
#   for word in words:
#     word = word.lower()
#     if word in index:
#       index[word] += 1
#     else:
#       index[word] = 1
#   return index

In [57]:
# from nltk.stem import PorterStemmer
# def apply_stemming_to_index(index):
#     stemmer = PorterStemmer()
#     stemmed_index = {}

#     for word, doc_ids in index.items():
#         stemmed_word = stemmer.stem(word)

#         if stemmed_word in stemmed_index:
#             stemmed_index[stemmed_word].update(doc_ids)
#         else:
#             stemmed_index[stemmed_word] = set(doc_ids)


#     final_index = {
#         word: sorted(list(doc_ids))
#         for word, doc_ids in stemmed_index.items()
#     }

#     return final_index
# inverted_index = build_inverted_index_by_document(units)
# inverted_index = apply_stemming_to_index(inverted_index)




In [58]:
# def createIndex():
#   stemmer = PorterStemmer()
#   myIndex = {}

#   for word in ALL_PLANT_DISEASE_KEYWORDS:

#       normalized_word = word.lower()
#       normalized_word = stemmer.stem(normalized_word)


#       if normalized_word in inverted_index:
#           myIndex[word] = inverted_index[normalized_word]
#       else:
#           myIndex[word] = []




In [59]:
#required inputs
# Core Python
import re
import math

# HTTP and parsing
import requests
from bs4 import BeautifulSoup
#Date fetching related
import pandas as pd
import matplotlib.pyplot as plt

# NLP
from nltk.stem import PorterStemmer

# Firebase
!pip install firebase
from firebase import firebase

# UI (Jupyter / Colab)
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Optional OpenAI
from openai import OpenAI
from google.colab import userdata





In [60]:
#Firebase section
#connection to DB


DBkey="https://cloudprojectdb-5e84b-default-rtdb.asia-southeast1.firebasedatabase.app/"
FBconn = firebase.FirebaseApplication(DBkey,None)



#uploading documents
# ===============================
# SOURCE URLS
# ===============================
SOURCE_URLS = {
    "Colorado_State_Extension":
        "https://extension.colostate.edu/resource/molecular-methods-for-diagnosing-plant-diseases/",
    "APS_Plant_Disease_Diagnosis":
        "https://www.apsnet.org/edcenter/apsnetfeatures/Pages/PDDiagnosis.aspx",
    "Frontiers_Plant_Disease":
        "https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2018.01652/full",
        "Recent advances in plant disease detection: challenges and opportunities":"https://link.springer.com/article/10.1186/s13007-025-01450-0",
        "A lightweight and explainable CNN model for empowering plant disease diagnosis":"https://www.nature.com/articles/s41598-025-94083-1"
}
count=1;
#uplading data
for source in SOURCE_URLS:
  key=source
  value=SOURCE_URLS[source]

  data_to_upload={
      "URL":key,
      "source_Id":count,
      "title":value,
      }
  res = FBconn.put(
           url=f'Sources/',
           name=count,
           data=data_to_upload
    )
  count+=1




In [61]:
# ===============================
# LOAD INVERTED INDEX
# ===============================


def load_inverted_index_from_db():
    data = FBconn.get("/InvertedIndex", None)
    if not data:
        return {}

    index = {}

    for term, entry in data.items():
        if not isinstance(entry, dict):
            index[term] = []
            continue

        doc_ids = entry.get("DocIDs")

        # Case 1: DocIDs is dict  { "1": true, "2": true }
        if isinstance(doc_ids, dict):
            index[term] = [str(k) for k in doc_ids.keys()]

        # Case 2: DocIDs is list  [None, True, True, None, True]
        elif isinstance(doc_ids, list):
            index[term] = [
                str(i) for i, v in enumerate(doc_ids) if v
            ]

        else:
            index[term] = []

    return index
CACHED_INVERTED_INDEX = load_inverted_index_from_db()
CACHED_DOCUMENTS = FBconn.get("/Documents", None)

In [62]:
# ===============================
# OPTIONAL OpenAI
# ===============================

client = None
try:
    from openai import OpenAI
    from google.colab import userdata

    api_key = userdata.get("OPENAI_API_KEY")
    if api_key:
        client = OpenAI(api_key=api_key)
        print("OpenAI connected")
    else:
        print("OpenAI key not found - embeddings and LLM disabled")
except Exception:
    client = None
    print("OpenAI not available")

# ===============================
# CONFIG
# ===============================
STOPWORDS = {
    "the","and","of","in","to","for","with","on","by","from",
    "is","are","was","were","be","been","being",
    "how","what","why","when","where","which","who","whom",
    "a","an","as","at","it","this","that","these","those"
}

ALL_PLANT_DISEASE_KEYWORDS = [
    "plant","disease","diseases","detection","diagnosis",
    "pathogen","pathogens","symptom","symptoms",
    "leaf","leaves","root","crop","loss",
    "necrosis","spot","spots",
    "fungi","fungal","bacterial",
    "virus","viral","nematode",
    "microscopy","laboratory","molecular","pcr",
    "image","learning","model",

]

stemmer = PorterStemmer()
MAX_CHARS_PER_DOC = 12000

# ===============================
# PREPROCESS
# ===============================
def preprocess(text):
    text = (text or "").lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS and len(t) > 1]
    tokens = [stemmer.stem(t) for t in tokens]
    return tokens

# preprocess keywords ONCE
KEYWORDS_PROCESSED = {stemmer.stem(k.lower()): k for k in ALL_PLANT_DISEASE_KEYWORDS}

# ===============================
# FETCH + CLEAN WEB PAGE
# ===============================
def fetch_web_page_text(url, max_chars=MAX_CHARS_PER_DOC):
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers, timeout=20)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")
    for tag in soup(["script", "style", "nav", "header", "footer", "aside"]):
        tag.decompose()

    text = soup.get_text(separator=" ")
    text = re.sub(r"\s+", " ", text).strip()
    return text[:max_chars]

# ===============================
# BUILD DOCUMENTS AND UPLOAD TO DB
# ===============================
count = 1
for name, url in SOURCE_URLS.items():
    try:
        clean_text = fetch_web_page_text(url)
        data_to_upload = {
            "doc_id": count,
            "source_Id": count,
            "clean_text": clean_text,
            "text_length": len(clean_text)
        }
        FBconn.put(
            url="Documents",
            name=str(count),
            data=data_to_upload
        )
        count += 1
    except Exception as e:
        print(f"Failed loading {name}: {e}")

# ===============================
# BUILD INVERTED INDEX (KEYWORDS ONLY)
# ===============================

documents = CACHED_DOCUMENTS

inverted_index = {}

if isinstance(documents, dict):
    iterable = documents.items()
elif isinstance(documents, list):
    iterable = enumerate(documents)
else:
    iterable = []

for doc_id, doc_data in iterable:
    if not doc_data:
        continue

    text = doc_data.get("clean_text")
    if not text:
        continue

    text_tokens = set(preprocess(text))

    for kw_stem in KEYWORDS_PROCESSED:
        if kw_stem in text_tokens:
            inverted_index.setdefault(kw_stem, {})[str(doc_id)] = True

# ===============================
# SAVE INVERTED INDEX TO DB
# ===============================
for term, doc_id_map in inverted_index.items():
    index_entry = {
        "term": term,
        "DocIDs": doc_id_map
    }

    try:
        FBconn.put(
            url="InvertedIndex",
            name=term,
            data=index_entry
        )
    except Exception as e:
        print(f"Failed saving term '{term}': {e}")

print("Inverted index built and saved.")
print("Number of indexed terms:", len(inverted_index))



OpenAI connected
Inverted index built and saved.
Number of indexed terms: 23


In [63]:


# ===============================
# BUILD CHUNKS
# ===============================

chunks = []
chunk_id = 0

# documents can be list or dict
if isinstance(documents, list):
    iterable = enumerate(documents)
elif isinstance(documents, dict):
    iterable = documents.items()
else:
    iterable = []

for doc_key, doc_data in iterable:
    if not doc_data:
        continue

    text = doc_data.get("clean_text")
    if not text:
        continue

    sentences = re.split(r"[.!?]\s+", text)
    buffer = []

    for s in sentences:
        s = s.strip()
        if len(s) < 30:
            continue

        buffer.append(s)

        if len(buffer) == 3:
            chunks.append({
                "chunk_id": chunk_id,
                "doc_id": doc_key,
                "text": " ".join(buffer)
            })
            chunk_id += 1
            buffer = []

print("Chunks built:", len(chunks))


# ===============================
# EMBEDDINGS - DB CHECK
# ===============================

def embeddings_exist_in_db():
    data = FBconn.get("/Embeddings", None)
    return data is not None


# ===============================
# BUILD AND STORE EMBEDDINGS
# ===============================

def build_and_store_embeddings(chunks):
    if client is None:
        print("No OpenAI client - skipping embeddings")
        return

    texts = [c["text"] for c in chunks]

    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=texts
        )

        for c, e in zip(chunks, response.data):
            embedding_entry = {
                "chunk_id": c["chunk_id"],
                "doc_id": c["doc_id"],
                "embedding": e.embedding
            }

            FBconn.put(
                url="Embeddings",
                name=str(c["chunk_id"]),
                data=embedding_entry
            )

        print("Embeddings stored in DB:", len(chunks))

    except Exception as e:
        print("Embedding error:", e)


# ===============================
# OPTIONAL OFFLINE STEP
# ===============================

if not embeddings_exist_in_db():
    print("No embeddings found in DB")

    if client is not None:
        print("Building embeddings...")
        build_and_store_embeddings(chunks)
    else:
        print("OpenAI not available - skipping embeddings build")
else:
    print("Embeddings already exist in DB - skipping build")


# ===============================
# LOAD EMBEDDINGS FROM DB
# ===============================

def load_embeddings_from_db():
    data = FBconn.get("/Embeddings", None)
    if not data:
        return []

    embeddings = []
    for _, v in data.items():
        embeddings.append(v)

    return embeddings


# ===============================
# COSINE SIMILARITY
# ===============================

def cosine_sim(a, b):
    return sum(x*y for x, y in zip(a, b)) / (
        math.sqrt(sum(x*x for x in a)) *
        math.sqrt(sum(y*y for y in b))
    )


# ===============================
# RETRIEVE CHUNKS BY EMBEDDING
# ===============================

def retrieve_chunks_by_embedding(query, top_k=5):
    if client is None:
        return []

    chunk_embeddings = load_embeddings_from_db()
    if not chunk_embeddings:
        return []

    try:
        q_emb = client.embeddings.create(
            model="text-embedding-3-small",
            input=query
        ).data[0].embedding

        scored = []
        for c in chunk_embeddings:
            sim = cosine_sim(q_emb, c["embedding"])
            scored.append((sim, c))

        scored.sort(reverse=True, key=lambda x: x[0])
        return [c for _, c in scored[:top_k]]

    except Exception as e:
        print("Embedding retrieval error:", e)
        return []


Chunks built: 115
No embeddings found in DB
Building embeddings...
Embedding error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [64]:



# ===============================
# KEYWORD RETRIEVAL
# ===============================

# def retrieve_docs(query, inverted_index=None, mode="OR"):
#     terms = preprocess(query)
#     if not terms:
#         return []

#     if inverted_index is None:
#         inverted_index = load_inverted_index_from_db()

#     sets = []
#     for t in terms:
#         if t in inverted_index:
#             sets.append(set(inverted_index[t]))

#     if not sets:
#         return []

#     return [int(d) for d in set.intersection(*sets)] if mode == "AND" else [int(d) for d in set.union(*sets)]




def retrieve_docs(query, inverted_index, mode="OR"):
    query_terms = preprocess(query)

    valid_terms = [t for t in query_terms if t in inverted_index]
    if not valid_terms:
        return []

    sets = [set(inverted_index[t]) for t in valid_terms]

    return (
        [int(d) for d in set.intersection(*sets)]
        if mode == "AND"
        else [int(d) for d in set.union(*sets)]
    )

# ===============================
# OPENAI GENERATION
# ===============================

def generate_with_openai(query, context):
    if client is None:
        return None

    try:
        response = client.responses.create(
            model="gpt-4o-mini",
            input=[
                {"role": "system", "content": "You are a scientific assistant."},
                {"role": "user", "content": f"Question:\n{query}\n\nContext:\n{context}"}
            ],
            temperature=0.2
        )

        for item in response.output:
            for content in item.content:
                if content["type"] == "output_text":
                    return content["text"].strip()

        return None

    except Exception:
        return None


# ===============================
# EXTRACTIVE FALLBACK
# ===============================

def generate_extractive_answer(query, docs):
    query_terms = set(preprocess(query))
    selected = []

    for d in docs:
        doc = documents.get(int(d)) if isinstance(documents, dict) else documents[int(d)]
        if not doc:
            continue

        for s in re.split(r"[.!?]\s+", doc.get("clean_text", "")):
            s_tokens = set(preprocess(s))
            score = len(s_tokens & query_terms)

            #score = len(set(preprocess(s)) & query_terms)
            if score > 0 and 50 <= len(s) <= 350:
                selected.append((score, s))

    if not selected:
        return "No explanatory content was identified."

    selected.sort(reverse=True)
    answer = "Plant disease detection relies on multiple complementary approaches. "

    for _, s in selected[:3]:
        answer += s.strip() + " "

    # answer += "\n\nSources:\n"
    # for d in docs:
    #     answer += f"- {d}\n"

    return answer


# ===============================
# UI
# ===============================

title = widgets.HTML("<h3>VRAG Chat</h3>")

query_box = widgets.Text(
    placeholder="Ask a question about plant disease detection",
    layout=widgets.Layout(width="100%")
)

mode_toggle = widgets.ToggleButtons(
    options=["OR", "AND"],
    value="OR",
    description="Keyword:"
)

llm_toggle = widgets.ToggleButtons(
    options=["With OpenAI", "Without OpenAI"],
    value="With OpenAI",
    description="Answer:"
)

ask_btn = widgets.Button(description="Ask", button_style="primary")
out = widgets.Output()

def rag_answer(query):
    # 1.  semantic / embeddings
    # sem = retrieve_chunks_by_embedding(query)
    # if sem:
    #     context = " ".join(c["text"] for c in sem)
    #     ans = generate_with_openai(query, context) if llm_toggle.value == "With OpenAI" else None
    #     if ans:
    #         return ans

    # 2. keyword-based retrieval
    docs = retrieve_docs(query, CACHED_INVERTED_INDEX)


    if not docs:

        return "No relevant information was found for your query."

    # 3. OpenAI on documents
    # if llm_toggle.value == "With OpenAI":
    #     context = " ".join(
    #         documents[d]["clean_text"][:500]
    #         for d in docs[:2]
    #         if d in documents
    #     )
    #     ans = generate_with_openai(query, context)
    #     if ans:
    #         return ans

    # 4. extractive fallback
    return generate_extractive_answer(query, docs)

def on_ask(_):
    q = query_box.value.strip()
    with out:
        clear_output()
        if not q:
            print("Please enter a question.")
            return

        answer = rag_answer(q)
        print(answer)


# ask_btn.on_click(on_ask)
# query_box.on_submit(lambda _: on_ask(None))

# display(widgets.VBox([
#     title,
#     query_box,
#     widgets.HBox([mode_toggle, llm_toggle, ask_btn]),
#     out
# ]))




In [65]:
# # ==========================================
# # 🧩 1. DASHBOARD
# # ==========================================
# def build_dashboard():
#     plant_select = widgets.Dropdown(options=['🍅 Tomato (Greenhouse)', '🌿 Basil (Kitchen)', '🍋 Lemon Tree'], layout=widgets.Layout(width='250px'))
#     def metric_card(icon, label, value, unit, color):
#         return widgets.HTML(f"""<div class="pro-card" style="width: 200px; padding: 25px; text-align: center; margin: 10px;"><div style="font-size: 32px; margin-bottom: 10px;">{icon}</div><div style="color: #b0bec5; font-size: 11px; font-weight: 700; text-transform: uppercase;">{label}</div><div style="font-size: 28px; font-weight: 700; color: {color}; margin-top:5px;">{value}<small style="font-size:14px; color:#cfd8dc;">{unit}</small></div></div>""")

#     row1 = widgets.HBox([metric_card("💧", "Soil Moisture", 62, "%", "#0288d1"), metric_card("☀️", "Light Level", 850, "lx", "#fbc02d")])
#     row2 = widgets.HBox([metric_card("🌡️", "Temperature", 24, "°C", "#e64a19"), metric_card("☁️", "Humidity", 55, "%", "#7cb342")])
#     btn_water = widgets.ToggleButton(description='💧 Water', layout=widgets.Layout(width='120px', height='40px'))
#     btn_light = widgets.ToggleButton(description='💡 Lights', value=True, layout=widgets.Layout(width='120px', height='40px'))
#     controls = widgets.HBox([widgets.Label("Manual Override:"), btn_water, btn_light], layout=widgets.Layout(align_items='center', grid_gap='15px', margin='20px 0'))

#     graph_out = widgets.Output()
#     with graph_out:
#         plt.figure(figsize=(7, 2.5), dpi=100)
#         plt.plot(np.sin(np.linspace(0, 10, 50)) + 20, color='#2e7d32', lw=2, label='Temp')
#         plt.plot(np.cos(np.linspace(0, 10, 50)) + 60, color='#0288d1', lw=2, label='Humid')
#         plt.legend(frameon=False, fontsize=8)
#         plt.axis('off')
#         plt.title("24h History Trend", loc='left', fontsize=10, color='#90a4ae')
#         plt.tight_layout()
#         plt.show()

#     return widgets.VBox([widgets.HBox([widgets.Label("📍 Active Sensor:"), plant_select], layout=widgets.Layout(margin='0 0 20px 0')), widgets.VBox([row1, row2], layout=widgets.Layout(align_items='center')), widgets.HTML("<div style='height:20px'></div>"), graph_out, controls], layout=widgets.Layout(align_items='center', padding='30px', width='100%'))


In [66]:
# Dashboard that reads REAL sensor data from the CENTRAL SERVER (no Adafruit keys)
# Feeds available (per course): temperature, humidity, soil, json
# NOTE: The server may "sleep". The first request can take ~20–40 seconds (this is normal). :contentReference[oaicite:0]{index=0}

# ==========================================
# 🌱 SENSOR GLOBAL INPUTS (SIMULATION / IoT)
# ==========================================
soil  = 0   # %
temp = 0     # °C
hum = 0         # %
light_level = 10      # lx


# =========================
# 1) SERVER CONFIG + HELPERS
# =========================

BASE_URL = "https://server-cloud-v645.onrender.com"

def get_history(feed: str, limit: int):
    """
    Fetch the last 'limit' samples of a feed from the central server.
    Returns a JSON dict that contains a list under key 'data'.
    """
    r = requests.get(
        f"{BASE_URL}/history",
        params={"feed": feed, "limit": int(limit)},
        timeout=60
    )
    r.raise_for_status()
    return r.json()

def get_last_value(feed: str):
    """
    Fetch the most recent value (limit=1) for a feed.
    Returns the value as a string (or None if missing).
    """
    data = get_history(feed, 1)
    if "data" in data and len(data["data"]) > 0:
        return data["data"][0].get("value", None)
    return None


# =========================
# 2) DASHBOARD UI
# =========================

def build_dashboard():
    # --- Plant selector (UI-only, not connected to server in this example) ---
    plant_select = widgets.Dropdown(
        options=['🍅 Tomato (Greenhouse)', '🌿 Basil (Kitchen)', '🍋 Lemon Tree'],
        layout=widgets.Layout(width='250px')
    )

    # --- A "metric card" that we can update later ---
    def metric_card(icon, label, unit, color, initial="--"):
        card = widgets.HTML()

        def set_value(v):
            card.value = f"""
            <div class="pro-card" style="width: 200px; padding: 25px; text-align: center; margin: 10px;">
              <div style="font-size: 32px; margin-bottom: 10px;">{icon}</div>
              <div style="color: #b0bec5; font-size: 11px; font-weight: 700; text-transform: uppercase;">{label}</div>
              <div style="font-size: 28px; font-weight: 700; color: {color}; margin-top:5px;">
                {v}<small style="font-size:14px; color:#cfd8dc;">{unit}</small>
              </div>
            </div>
            """

        set_value(initial)
        card.set_value = set_value  # attach setter so we can update the card later
        return card

    # --- Create cards (start with "--" until we fetch real data) ---
    soil_card = metric_card("💧", "Soil Moisture", "%", "#0288d1")
    light_card = metric_card("☀️", "Light Level", "lx", "#fbc02d")
    temp_card = metric_card("🌡️", "Temperature", "°C", "#e64a19")
    hum_card  = metric_card("☁️", "Humidity", "%", "#7cb342")

    row1 = widgets.HBox([soil_card, light_card])
    row2 = widgets.HBox([temp_card, hum_card])

    # --- Manual override toggles (UI-only, not connected to hardware in this example) ---
    btn_water = widgets.ToggleButton(description='💧 Water', layout=widgets.Layout(width='120px', height='40px'))
    btn_light = widgets.ToggleButton(description='💡 Lights', value=True, layout=widgets.Layout(width='120px', height='40px'))
    controls = widgets.HBox(
        [widgets.Label("Manual Override:"), btn_water, btn_light],
        layout=widgets.Layout(align_items='center', grid_gap='15px', margin='20px 0')
    )

    # --- Controls for plotting history ---
    limit_slider = widgets.IntSlider(
        value=50, min=5, max=200, step=5,
        description="History:"
    )

    refresh_btn = widgets.Button(description="🔄 Refresh", button_style="success")
    status = widgets.HTML("<span style='color:#90a4ae'>Ready</span>")

    graph_out = widgets.Output()

    # =========================
    # 3) REFRESH FUNCTION
    # =========================

    def refresh(_=None):
        """
        1) Fetch latest values for cards (temperature/humidity/soil)
        2) Fetch last N samples for temperature and humidity and plot them
        """
        status.value = "<span style='color:#90a4ae'>Fetching data... (first request can be slow)</span>"

        # ---- Update cards with the LAST value (limit=1) ----
        try:
            temp = get_last_value("temperature")
            hum  = get_last_value("humidity")
            soil = get_last_value("soil")

            temp_card.set_value(temp if temp is not None else "--")
            hum_card.set_value(hum if hum is not None else "--")
            soil_card.set_value(soil if soil is not None else "--")

            # Light feed is not provided by course server ,keep it "--"
            light_card.set_value("--")

        except Exception as e:
            status.value = f"<span style='color:#c62828'>Card update error: {e}</span>"
            return

        # ---- Plot history (last N) for Temp + Humidity ----
        with graph_out:
            clear_output()
            try:
                N = int(limit_slider.value)

                t_hist = get_history("temperature", N).get("data", [])
                h_hist = get_history("humidity", N).get("data", [])

                tdf = pd.DataFrame(t_hist)
                hdf = pd.DataFrame(h_hist)

                # Convert time + numeric values
                if "created_at" in tdf.columns:
                    tdf["created_at"] = pd.to_datetime(tdf["created_at"], errors="coerce")
                if "created_at" in hdf.columns:
                    hdf["created_at"] = pd.to_datetime(hdf["created_at"], errors="coerce")

                if "value" in tdf.columns:
                    tdf["value_num"] = pd.to_numeric(tdf["value"], errors="coerce")
                if "value" in hdf.columns:
                    hdf["value_num"] = pd.to_numeric(hdf["value"], errors="coerce")

                tdf = tdf.dropna(subset=["created_at", "value_num"]).sort_values("created_at")
                hdf = hdf.dropna(subset=["created_at", "value_num"]).sort_values("created_at")

                plt.figure(figsize=(7, 2.5), dpi=100)

                if len(tdf) > 0:
                    plt.plot(tdf["created_at"], tdf["value_num"], lw=2, label="Temp")
                if len(hdf) > 0:
                    plt.plot(hdf["created_at"], hdf["value_num"], lw=2, label="Humid")

                plt.legend(frameon=False, fontsize=8)
                plt.title("24h History Trend (Real Data)", loc="left", fontsize=10, color="#90a4ae")
                plt.xticks(rotation=30, fontsize=7)
                plt.tight_layout()
                plt.show()

                status.value = "<span style='color:#2e7d32'>Updated ✅</span>"

            except Exception as e:
                status.value = f"<span style='color:#c62828'>Plot error: {e}</span>"

    # Button click => refresh
    refresh_btn.on_click(refresh)

    # Top controls
    top_controls = widgets.HBox(
        [refresh_btn, status, limit_slider],
        layout=widgets.Layout(margin='0 0 10px 0')
    )

    # Do an initial refresh automatically
    refresh()

    # Final dashboard layout
    return widgets.VBox(
        [
            widgets.HBox(
                [widgets.Label("📍 Active Sensor:"), plant_select],
                layout=widgets.Layout(margin='0 0 20px 0')
            ),
            top_controls,
            widgets.VBox([row1, row2], layout=widgets.Layout(align_items='center')),
            widgets.HTML("<div style='height:20px'></div>"),
            widgets.HTML("<div style='color:#90a4ae; font-size:12px; width:100%; max-width:720px;'>24h History Trend</div>"),
            graph_out,
            controls
        ],
        layout=widgets.Layout(align_items='center', padding='30px', width='100%')
    )


In [67]:
from numbers import Number
# ==========================================
# 🌱 SENSOR INPUTS (SIMULATION / IoT)
# ==========================================

temp = get_last_value("temperature")
hum  = get_last_value("humidity")
soil = get_last_value("soil")

soil_moisture = soil     # %
temperature = temp      # °C
humidity = hum         # %
light_level = 10      # lx
# ==========================================
# ⚠️ RISK FUNCTIONS PER PARAMETER
# 0 = OK | 1 = Warning | 2 = Critical
# ==========================================
def water_risk(m):
    if m < 40:
        return 2
    elif m < 60:
        return 1
    return 0

def heat_risk(t):
    if t < 10 or t > 35:
        return 2
    elif t < 15 or t > 30:
        return 1
    return 0

def humidity_risk(h):
    if h > 85:
        return 2
    elif h > 70:
        return 1
    return 0

def light_risk(l):
    if l < 300:
        return 2
    elif l < 600:
        return 1
    return 0


# ==========================================
# 🌿 OVERALL PLANT STATE
# ==========================================
def compute_plant_state(soil, temp, humid, light):
    print(soil,temp,humid,light)

    risks = {
        "Water": water_risk(float(soil)),
        "Heat": heat_risk(float(temp)),
        "Humidity": humidity_risk(float(humid)),
        "Light": light_risk(float(light))
    }

    if 2 in risks.values():
        return risks, "Critical", "#d32f2f", "Immediate attention required"
    elif sum(risks.values()) >= 2:
        return risks, "Warning", "#f9a825", "Plant conditions are unstable"
    else:
        return risks, "Healthy", "#2e7d32", "Plant conditions are optimal"


risks, plant_state, state_color, state_note = compute_plant_state(
    soil_moisture, temperature, humidity, light_level
)


# ==========================================
# 🎨 UI COMPONENTS
# ==========================================
def risk_card(icon, title, level, note):
    colors = {0: "#2e7d32", 1: "#f9a825", 2: "#d32f2f"}
    labels = {0: "Low", 1: "Medium", 2: "High"}

    return widgets.HTML(f"""
    <div class="pro-card" style="
        width: 220px;
        padding: 20px;
        text-align: center;
        border-top: 5px solid {colors[level]};
    ">
        <div style="font-size: 28px;">{icon}</div>
        <div style="font-weight: 600; margin-top: 8px;">{title}</div>
        <div style="font-size: 22px; font-weight: 700; color:{colors[level]};">
            {labels[level]}
        </div>
        <div style="font-size: 12px; color:#546e7a; margin-top:6px;">
            {note}
        </div>
    </div>
    """)

def build_plant_state_tab():

  # ==========================================
  # 🧩 RISK CARDS ROW
  # ==========================================
  risk_cards = widgets.HBox([
      risk_card("💧", "Water Stress", risks["Water"], "Soil moisture level"),
      risk_card("🌡️", "Heat Stress", risks["Heat"], "Temperature stability"),
      risk_card("🍄", "Disease Risk", risks["Humidity"], "Humidity impact"),
      risk_card("☀️", "Light Condition", risks["Light"], "Light availability")
  ], layout=widgets.Layout(justify_content="center", gap="15px"))
  # ==========================================
  # 🌱 PLANT STATE SUMMARY CARD
  # ==========================================
  plant_state_card = widgets.HTML(f"""
  <div class="pro-card" style="
      padding: 30px;
      text-align: center;
      border-left: 6px solid {state_color};
      max-width: 500px;
      margin: 30px auto;
  ">
      <div style="font-size: 30px; font-weight: 700; color:{state_color};">
          🌱 Plant State: {plant_state}
      </div>
      <div style="margin-top: 10px; color:#546e7a; font-size:14px;">
          {state_note}
      </div>
  </div>
  """)

  return widgets.VBox([
      plant_state_card,
      risk_cards])






  20 21.50 36.00 10


In [68]:
# ==========================================
# 🧩 2. AI DIAGNOSIS (With Image Fix)
# ==========================================
def build_ai_tab():
    how_it_works = widgets.HTML("""<div style="display: flex; justify-content: center; gap: 40px; margin-bottom: 30px; opacity: 0.8;"><div style="text-align: center;"><div style="font-size: 24px; background: #e8f5e9; width: 50px; height: 50px; line-height: 50px; border-radius: 50%; margin: auto; color: #2e7d32;">1</div><div style="font-size: 12px; font-weight: bold; margin-top: 8px; color: #546e7a;">UPLOAD</div></div><div style="text-align: center;"><div style="font-size: 24px; background: #e3f2fd; width: 50px; height: 50px; line-height: 50px; border-radius: 50%; margin: auto; color: #0277bd;">2</div><div style="font-size: 12px; font-weight: bold; margin-top: 8px; color: #546e7a;">ANALYZE</div></div><div style="text-align: center;"><div style="font-size: 24px; background: #fff3e0; width: 50px; height: 50px; line-height: 50px; border-radius: 50%; margin: auto; color: #ef6c00;">3</div><div style="font-size: 12px; font-weight: bold; margin-top: 8px; color: #546e7a;">RESULTS</div></div></div>""")

    # We add the class 'upload-zone' to trigger the CSS above
    uploader = widgets.FileUpload(accept='image/*', multiple=False)
    uploader.add_class('upload-zone')

    preview = widgets.Image(width=300, height=300, layout=widgets.Layout(display='none', margin='20px auto', border='5px solid white', box_shadow='0 4px 15px rgba(0,0,0,0.1)', object_fit='cover'))
    output = widgets.Output()

    def on_upload(change):
        if not uploader.value: return
        output.clear_output()

        file_info = list(uploader.value.values())[0]
        preview.value = file_info['content']
        preview.layout.display = 'block'

        with output:
            display(HTML("""<div class="pro-card" style="padding: 20px; margin-top: 20px; border-left: 5px solid #2e7d32;"><h3 style="margin: 0; color: #2e7d32;">✅ Healthy Plant Detected</h3><p style="margin: 5px 0; color: #546e7a;">Our AI is <strong>98.5%</strong> confident.</p><div style="margin-top: 10px; font-size: 12px; background: #f1f8e9; padding: 8px; border-radius: 8px; color: #33691e;">💡 Tip: Keep maintaining your current watering schedule.</div></div>"""))

    uploader.observe(on_upload, names='value')

    return widgets.VBox([
        widgets.HTML("<h2 style='text-align:center; color:#37474f; margin-bottom:5px;'>AI Plant Doctor</h2>"),
        widgets.HTML("<p style='text-align:center; color:#90a4ae; margin-bottom:30px;'>Detect diseases early with computer vision.</p>"),
        how_it_works,
        widgets.Box([uploader], layout=widgets.Layout(width='500px', margin='0 auto')),
        preview,
        output
    ], layout=widgets.Layout(padding='40px', align_items='center', width='100%'))

In [69]:
# ==========================================
# 🧩 3. SEARCH TAB
# ==========================================

def build_search_tab():
    header = widgets.HTML("""
        <div style="text-align: center; margin-bottom: 30px;">
            <h2 style="color: #37474f; font-size: 28px; margin-bottom: 10px;">
                Knowledge Base
            </h2>
            <p style="color: #90a4ae;">
                Find care guides, pest solutions, and expert tips.
            </p>
        </div>
    """)

    search_bar = widgets.Text(
        placeholder='e.g., "Why are my leaves yellow?"',
        layout=widgets.Layout(width='500px')
    )
    search_bar.add_class('hero-search')

    def chip(text):
        btn = widgets.Button(description=text)
        btn.add_class('tag-chip')
        return btn

    tags = widgets.HBox(
        [chip('🍅 Tomatoes'), chip('🐛 Pests'), chip('💧 Watering'), chip('🍂 Fertilizing')],
        layout=widgets.Layout(justify_content='center', margin='15px 0')
    )

    results_area = widgets.Output()


    # RAG connection
    def on_search_submit(text):
        query = text.value.strip()
        if not query:
            return

        with results_area:
            clear_output()
            display(HTML("""
                <div class="pro-card" style="padding:20px; max-width:600px;">
                    <div style="color:#90a4ae; font-size:12px;">
                        Searching knowledge base...
                    </div>
                </div>
            """))

        # Call RAG
        answer = rag_answer(query)
        if not answer:
            answer = "No relevant information was found for your query."


        with results_area:
            clear_output()
            display(HTML(f"""
                <div class="pro-card" style="padding:20px; max-width:600px;">
                    <div style="color:#0277bd; font-weight:bold; margin-bottom:10px;">
                        Answer
                    </div>
                    <div style="color:#37474f; line-height:1.7;">
                        {answer.replace(chr(10), '<br>')}
                    </div>
                </div>
            """))

    # search only when hitting enter
    search_bar.on_submit(on_search_submit)

    return widgets.VBox(
        [header, search_bar, tags, results_area],
        layout=widgets.Layout(padding='40px', align_items='center')
    )


In [70]:
# ==========================================
# 🧩 4. LEADERBOARD
# ==========================================
def build_leaderboard():
    rows = ""
    data = [(1, "Charlie 👑", 1250), (2, "Alice", 980), (3, "Bob", 850), (4, "You", 320)]
    for r, n, s in data:
        bg = "#f1f8e9" if r == 1 else "white"
        rows += f"<tr style='background:{bg}; border-bottom:1px solid #eee;'><td style='padding:15px;'>#{r}</td><td style='padding:15px;'>{n}</td><td style='padding:15px; text-align:right; font-weight:bold; color:#2e7d32;'>{s} pts</td></tr>"
    return widgets.HTML(f"""<div style="max-width: 500px; margin: auto;"><h2 style="text-align:center; color:#37474f;">🏆 Top Gardeners</h2><div class="pro-card" style="overflow: hidden; padding: 0;"><table style="width:100%; border-collapse:collapse; font-size: 14px;">{rows}</table></div></div>""")


In [71]:
# @title 🌿 Smart Garden OS (Visual Bug Fixes)
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import numpy as np

# ==========================================
# 🎨 1. PROFESSIONAL CSS (Fixed)
# ==========================================
style = """
<style>
    /* GLOBAL THEME */
    :root {
        --primary: #2e7d32;      /* Forest Green */
        --bg-color: #f9fbfd;
        --text-dark: #37474f;
    }

    /*  TABS */
    .widget-tab > .p-TabBar .p-TabBar-tab {
        background: transparent !important; border: none !important;
        border-bottom: 3px solid transparent !important;
        font-family: 'Roboto', sans-serif !important;
        font-weight: 600 !important;
        padding: 12px 20px !important;
        color: #90a4ae !important;
        font-size: 14px !important;
        flex: 1 1 auto !important;
        min-width: fit-content !important;
        white-space: nowrap !important;
        text-align:center !important

    }
    .widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current {
        color: var(--primary) !important; border-bottom: 3px solid var(--primary) !important;
    }

    /* 🔍 SEARCH BAR */
    .hero-search input {
        border-radius: 50px !important; border: 1px solid #eceff1 !important;
        padding: 20px 25px !important; font-size: 16px !important;
        box-shadow: 0 5px 15px rgba(0,0,0,0.05) !important;
    }

    /* 🏷️ TAG CHIPS */
    .tag-chip {
        background: white !important; border: 1px solid #eceff1 !important;
        border-radius: 20px !important; padding: 5px 15px !important;
        margin: 5px !important; color: #546e7a !important;
    }

    /*  UPLOAD ZONE (Force Big Icon) */
    /* We target the button broadly to ensure Colab applies it */
    .upload-zone button {
        background-color: #fafafa !important;

        /* Hide the default "Upload (0)" text */
        color: transparent !important;
        font-size: 0px !important;

        /* Make it big */
        height: 150px !important;
        width: 100% !important;
        border: 2px dashed #cfd8dc !important;
        border-radius: 20px !important;

        /* Add Background Icon */
        background-image: url("https://cdn-icons-png.flaticon.com/512/3097/3097412.png") !important;
        background-repeat: no-repeat !important;
        background-position: center 35% !important;
        background-size: 50px !important;
        transition: all 0.3s ease;
    }

    /* Add Custom Text Label */
    .upload-zone button::after {
        content: 'Click or Drag Leaf Photo Here';
        color: #78909c !important;
        font-size: 16px !important; /* Restore font size for label */
        font-weight: bold !important;
        position: absolute;
        top: 65% !important; left: 50% !important;
        transform: translate(-50%, 0) !important;
        pointer-events: none !important;
    }

    .upload-zone button:hover {
        border-color: var(--primary) !important;
        background-color: #f1f8e9 !important;
    }

    /* CARDS */
    .pro-card {
        background: white; border-radius: 16px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.03);
        border: 1px solid #f0f0f0; transition: transform 0.2s;
    }
    .pro-card:hover { transform: translateY(-3px); }
</style>
"""
display(HTML(style))






# ==========================================
# 🚀 APP ASSEMBLY
# ==========================================

app = widgets.Tab([
    build_dashboard(),
    build_plant_state_tab(),
    build_ai_tab(),
    build_search_tab(),
    build_leaderboard()
])

app.set_title(0, "📊 Dashboard")
app.set_title(1, "🌱 Plant State")
app.set_title(2, "📷 AI Diagnosis")
app.set_title(3, "🔍 Search")
app.set_title(4, "🏆 Leaderboard")



display(HTML("<div style='text-align:center; margin-bottom: 10px;'><h1 style='color:#2e7d32; font-family:sans-serif; margin:0;'>🌿 Smart Garden OS</h1><span style='font-size:12px; color:#90a4ae; letter-spacing: 2px;'>PROFESSIONAL EDITION</span></div>"))
display(app)